In [46]:
%%writefile requirments.txt
datasets
trl
peft

Writing requirments.txt


In [47]:
!pip install -r requirments.txt

In [48]:
from datasets import load_dataset
from transformers import AutoTokenizer
from peft import LoraConfig
from transformers import AutoModelForCausalLM
import torch
import torch.nn as nn
from trl import SFTTrainer
from transformers import TrainingArguments
from peft import PeftModel
from transformers import AutoModelForCausalLM

In [3]:
ds = load_dataset("FinGPT/fingpt-sentiment-train", split="train")
print(ds[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/529 [00:00<?, ?B/s]

data/train-00000-of-00001-dabab110260ac9(…):   0%|          | 0.00/6.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/76772 [00:00<?, ? examples/s]

{'input': 'Teollisuuden Voima Oyj , the Finnish utility known as TVO , said it shortlisted Mitsubishi Heavy s EU-APWR model along with reactors from Areva , Toshiba Corp. , GE Hitachi Nuclear Energy and Korea Hydro & Nuclear Power Co. .', 'output': 'neutral', 'instruction': 'What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.'}


In [4]:
def sample_text(example):
    text = (
        f"{example['instruction']}\n"
        f"Content: {example['input']}\n"
        f"Sentiment: {example['output']}"
    )
    return {"text": text}


In [5]:
ds = ds.map(sample_text, remove_columns=["instruction", "input", "output"])
print(next(iter(ds["text"])))

Map:   0%|          | 0/76772 [00:00<?, ? examples/s]

What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Content: Teollisuuden Voima Oyj , the Finnish utility known as TVO , said it shortlisted Mitsubishi Heavy s EU-APWR model along with reactors from Areva , Toshiba Corp. , GE Hitachi Nuclear Energy and Korea Hydro & Nuclear Power Co. .
Sentiment: neutral


In [6]:
ds = ds.train_test_split(test_size=0.2, seed=42)
ds_train = ds["train"]
ds_eval = ds["test"]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [13]:
lora_config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
)

In [16]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b", torch_dtype = torch.float16, device_map="auto")

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [18]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float16)

model.gradient_checkpointing_enable
model.enable_input_require_grads

class OutputToFloat(nn.Sequential):
  def forward(self, x):
    return super().forward(x).to(torch.float16)

model.lm_head = OutputToFloat(model.lm_head)

In [50]:
training_args = TrainingArguments(
    output_dir="./OPT-fine_tuned-FinGPT-CPU",
    dataloader_drop_last=True,
    max_steps = 10,
    save_strategy="steps",
    num_train_epochs=10,
    logging_steps=5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    fp16=False,
    bf16=True,
    weight_decay=0.05,
    ddp_find_unused_parameters=False,
    run_name="OPT-fine_tuned-FinGPT-CPU",
    report_to="wandb",
)

In [51]:
trainer = SFTTrainer(
    model = model,
    args = training_args,
    train_dataset = ds_train,
    eval_dataset = ds_eval,
    peft_config = lora_config,
)

print("Training...")
trainer.train()

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Training...


Step,Training Loss
5,3.888400
10,3.666400


wandb: WARNING URL not available in offline run


TrainOutput(global_step=10, training_loss=3.7774057388305664, metrics={'train_runtime': 26.6571, 'train_samples_per_second': 4.502, 'train_steps_per_second': 0.375, 'total_flos': 82447357870080.0, 'train_loss': 3.7774057388305664, 'epoch': 0.0019538882375928096})

In [34]:
model = AutoModelForCausalLM.from_pretrained(
  "facebook/opt-1.3b", return_dict=True, torch_dtype=torch.bfloat16
)

In [43]:
model = PeftModel.from_pretrained(model, "./OPT-fine_tuned-FinGPT-CPU/checkpoint-10/")
model.eval()
model = model.merge_and_unload()
model.save_pretrained("./OPT-fine_tuned-FinGPT-CPU/merged")